In [ ]:
import pandas as pd
import numpy as np
import keras
import h5py
#import resnet_101_keras.py
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras import applications
from keras.applications import ResNet50
#from resnet_101_keras import resnet101_model
from keras.applications.inception_v3 import InceptionV3
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
import keras_metrics
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
import os, sys

from keras.applications import VGG19
from keras.applications.vgg19 import preprocess_input
from keras import optimizers

trainLabelsPath = "traininglabels.csv"
trainImagesPath = "/Users/HS/Downloads/widsdatathon2019/train_images/"

testFolderPath = "/Users/HS/Downloads/widsdatathon2019/leaderboard_test_data"
testImagesPath = "/Users/HS/Downloads/widsdatathon2019/leaderboard_test_data/"

holdFolderPath = "/Users/HS/Downloads/widsdatathon2019/leaderboard_holdout_data"
holdImagesPath = "/Users/HS/Downloads/widsdatathon2019/leaderboard_holdout_data/"

subFilePath = "finalSubmission_underSamp.csv"

In [0]:

trainLabelsPath = "/home/ubuntu/WIDS/traininglabels.csv"
trainImagesPath = "/home/ubuntu/WIDS/train_images/"

testFolderPath = "/home/ubuntu/WIDS/leaderboard_test_data"
testImagesPath = "/home/ubuntu/WIDS/leaderboard_test_data/"

holdFolderPath = "/home/ubuntu/WIDS/leaderboard_holdout_data"
holdImagesPath = "/home/ubuntu/WIDS/leaderboard_holdout_data/"

subFilePath = "finalSubmission_underSamp_VGG19_dummy_15freez.csv"

# Upload training data file

In [0]:
df = pd.read_csv(trainLabelsPath)

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15244 entries, 0 to 15243
Data columns (total 3 columns):
image_id       15244 non-null object
has_oilpalm    15244 non-null int64
score          15244 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 357.4+ KB


In [0]:
df[df.has_oilpalm == 1].count() # only 6% of the images have palm oil plantations

image_id       942
has_oilpalm    942
score          942
dtype: int64

In [0]:
df.score.sort_values()[:5]
# some images have very small scores. Perhaps use a threshold like 0.5 and give 0.5 or less, label 0

1062    0.3887
72      0.3944
353     0.3954
1404    0.3963
270     0.3965
Name: score, dtype: float64

In [0]:
df = df.sort_values(by='image_id').reset_index(drop=True)

In [0]:
df_hasPalm = df[df.has_oilpalm == 1]
df_noPalm = df[(df.has_oilpalm == 0) & (df.score == 1)]

In [0]:
randomstate=0
size=1000
df_noPalm_samp = df_noPalm.sample(n=size, random_state=randomstate)
df_noPalm_samp.shape

(1000, 3)

In [0]:
df_final = df_hasPalm.append(df_noPalm_samp).sort_values(by='image_id').reset_index(drop=True)

In [0]:
df_final.head()

,image_id,has_oilpalm,score
0,img_000082017.jpg,0,1.0
1,img_000152017.jpg,0,1.0
2,img_000192018.jpg,0,1.0
3,img_000282018.jpg,0,1.0
4,img_000322017.jpg,0,1.0


In [0]:
type(df_final.image_id)

pandas.core.series.Series

In [0]:
# Reading images from a folder and converting to numpy array
train_image = []
for i in tqdm(range(len(df_final))):
    img = image.load_img(trainImagesPath + df_final.image_id[i], target_size=(224,224,1))
    img = image.img_to_array(img)
    img = img/255
    
    
    train_image.append(img)
X = np.array(train_image)
X





  0%|          | 0/1942 [00:00<?, ?it/s]



  0%|          | 1/1942 [00:07<3:57:09,  7.33s/it]



  0%|          | 2/1942 [00:07<2:49:49,  5.25s/it]



  0%|          | 3/1942 [00:08<2:02:41,  3.80s/it]



  0%|          | 4/1942 [00:08<1:28:23,  2.74s/it]



  0%|          | 5/1942 [00:08<1:04:37,  2.00s/it]



  0%|          | 6/1942 [00:08<47:54,  1.48s/it]  



  0%|          | 7/1942 [00:09<36:23,  1.13s/it]



  0%|          | 8/1942 [00:10<39:49,  1.24s/it]



  0%|          | 9/1942 [00:11<30:27,  1.06it/s]



  1%|          | 10/1942 [00:11<24:02,  1.34it/s]



  1%|          | 11/1942 [00:11<19:24,  1.66it/s]



  1%|          | 12/1942 [00:11<16:23,  1.96it/s]



  1%|          | 13/1942 [00:12<14:09,  2.27it/s]



  1%|          | 14/1942 [00:12<12:31,  2.57it/s]



  1%|          | 15/1942 [00:12<11:18,  2.84it/s]



  1%|          | 16/1942 [00:12<10:14,  3.13it/s]



  1%|          | 17/1942 [00:13<09:30,  3.37it/s]



  1%|          | 18/1942 [00:13<09:23,  3.42it/s

array([[[[0.08235294, 0.13333334, 0.05490196],
         [0.07058824, 0.12156863, 0.05490196],
         [0.05490196, 0.10980392, 0.05098039],
         ...,
         [0.01568628, 0.08235294, 0.01176471],
         [0.01568628, 0.08235294, 0.01176471],
         [0.06666667, 0.13333334, 0.0627451 ]],

        [[0.08235294, 0.13333334, 0.05490196],
         [0.04313726, 0.09411765, 0.02352941],
         [0.01568628, 0.07058824, 0.01176471],
         ...,
         [0.01960784, 0.08627451, 0.01568628],
         [0.02352941, 0.09019608, 0.01960784],
         [0.0627451 , 0.12941177, 0.05882353]],

        [[0.05098039, 0.10588235, 0.01568628],
         [0.00784314, 0.05882353, 0.        ],
         [0.        , 0.04705882, 0.        ],
         ...,
         [0.01960784, 0.08627451, 0.01568628],
         [0.04313726, 0.09803922, 0.03137255],
         [0.0627451 , 0.11764706, 0.05098039]],

        ...,

        [[0.04313726, 0.11764706, 0.04313726],
         [0.00784314, 0.08235294, 0.00392157]

In [0]:
y = df_final['has_oilpalm'].values
#y = to_categorical(y)
y

array([0, 0, 0, ..., 0, 0, 0])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2,stratify=y)

In [0]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1553, 256, 256, 3), (389, 256, 256, 3), (1553,), (389,))

In [ ]:
model = Sequential()

model.add(VGG19(include_top=True,pooling=None,weights='imagenet', input_shape= (224,224,3)))

model.summary()

In [ ]:
# freezing all layers

for layer in model.layers:
    layer.trainable = False

In [ ]:
x = model.layers[-2].output

prediction = Dense(1, activation="sigmoid")(x)

In [ ]:
# final model

model_final = Model(input=model.input, output = prediction)

In [ ]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
#adam = Adam(lr=0.0001)
model_final.compile(optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy',metrics = [keras_metrics.precision(), keras_metrics.recall(),'accuracy'])

In [0]:
#base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (150,150,3))

In [0]:
#x = base_model.output
#x = GlobalAveragePooling2D()(x)
#x = Dropout(0.7)(x)
#predictions = Dense(2, activation= 'sigmoid')(x)
#model = Model(inputs = base_model.input, outputs = predictions)

In [0]:
# final model

#model_final = Model(input=model.input, output = prediction)
#model.add(ResNet50(weights='imagenet', include_top=False,input_shape= (256,256,3)))

In [0]:
#model = Sequential()
#model.add(InceptionV3(weights='imagenet', include_top=False,input_shape= (256,256,3)))
#model.add(BatchNormalization())
#model.add(GlobalAveragePooling2D())
#model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [ ]:
#base_model = InceptionV3(weights='imagenet', include_top=False,input_shape= (256,256,3))
#x = base_model.output
#x = BatchNormalization()(x)
#x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)
#x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)

#output = Dense(1, activation='sigmoid')(x)
#model = Model(input=base_model.input, output=output)

In [0]:
#for layer in model.layers[:25]:
    #layer.trainable = False


#last_layer = model.output

# add a global spatial average pooling layer
#x = GlobalAveragePooling2D()(last_layer)

# Add fully connected & dropout layers
#x = Dense(512, activation='relu', name='fc-1')(x)
#x = Dropout(0.5)(x)
#x = Dense(256, activation='relu', name='fc-2')(x)
#x = Dropout(0.5)(x)

# softmax layer for 5 classes
#out = Dense(1, activation='sigmoid', name='output_layer')(x)

#model_final = Model(model.input, outputs=out)

    

In [0]:
#from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
#adam = Adam(lr=0.0001)
#model.compile(optimizer=adam, loss='binary_crossentropy',metrics = [keras_metrics.precision(), keras_metrics.recall(),'accuracy'])

from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class roc_callback(Callback):
    def __init__(self,training_data,validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model_final.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model_final.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        print('\rroc-auc: %s - roc-auc_val: %s' % (str(round(roc,4)),str(round(roc_val,4))),end=100*' '+'\n')
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

# Build model with training data and get loss and accuracy scores
model_final.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test),callbacks=[roc_callback(training_data=(X_train, y_train),validation_data=(X_test, y_test))])

In [ ]:
model_final.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

In [ ]:
model_final.save('Vgg_dummy_15freez.h5')

# Leaderboard Test Data Prediction

In [0]:
testpath = testFolderPath
testimagesList = sorted(os.listdir(testpath))
testimagesList

['img_000022018.jpg',
 'img_000042017.jpg',
 'img_000052017.jpg',
 'img_000062017.jpg',
 'img_000062018.jpg',
 'img_000122017.jpg',
 'img_000142018.jpg',
 'img_000162018.jpg',
 'img_000182017.jpg',
 'img_000262017.jpg',
 'img_000292017.jpg',
 'img_000302018.jpg',
 'img_000342018.jpg',
 'img_000382017.jpg',
 'img_000382018.jpg',
 'img_000422017.jpg',
 'img_000422018.jpg',
 'img_000432017.jpg',
 'img_000472018.jpg',
 'img_000532017.jpg',
 'img_000542018.jpg',
 'img_000552017.jpg',
 'img_000592017.jpg',
 'img_000602018.jpg',
 'img_000622017.jpg',
 'img_000622018.jpg',
 'img_000632018.jpg',
 'img_000672018.jpg',
 'img_000682018.jpg',
 'img_000692017.jpg',
 'img_000702017.jpg',
 'img_000752017.jpg',
 'img_000762017.jpg',
 'img_000812018.jpg',
 'img_000822018.jpg',
 'img_000832018.jpg',
 'img_000842018.jpg',
 'img_000852018.jpg',
 'img_000872018.jpg',
 'img_000882017.jpg',
 'img_000882018.jpg',
 'img_000902018.jpg',
 'img_000912017.jpg',
 'img_000912018.jpg',
 'img_000922018.jpg',
 'img_0009

In [0]:
leaderboard_test_pred = pd.DataFrame(testimagesList , columns=['image_id'])
leaderboard_test_pred

,image_id
0,img_000022018.jpg
1,img_000042017.jpg
2,img_000052017.jpg
3,img_000062017.jpg
4,img_000062018.jpg
5,img_000122017.jpg
6,img_000142018.jpg
7,img_000162018.jpg
8,img_000182017.jpg
9,img_000262017.jpg


In [0]:
test_image = []
for i in tqdm(range(len(testimagesList))):
    img = image.load_img(testImagesPath + testimagesList[i], target_size=(224,224,3))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)
test

100%|██████████| 4356/4356 [00:10<00:00, 402.52it/s]


array([[[[ 0.43529412,  0.35294119,  0.23137255],
         [ 0.52941179,  0.44705883,  0.33333334],
         [ 0.54901963,  0.46666667,  0.36078432],
         ..., 
         [ 0.18431373,  0.1882353 ,  0.13333334],
         [ 0.18039216,  0.18431373,  0.12941177],
         [ 0.30980393,  0.3137255 ,  0.25882354]],

        [[ 0.61960787,  0.53725493,  0.42352942],
         [ 0.58823532,  0.50588238,  0.39215687],
         [ 0.56078434,  0.47843137,  0.37254903],
         ..., 
         [ 0.17647059,  0.18431373,  0.12941177],
         [ 0.18431373,  0.1882353 ,  0.13333334],
         [ 0.21960784,  0.22352941,  0.16862746]],

        [[ 0.77254903,  0.67843139,  0.56862748],
         [ 0.68235296,  0.60000002,  0.48627451],
         [ 0.56470591,  0.48235294,  0.3764706 ],
         ..., 
         [ 0.16470589,  0.17254902,  0.12156863],
         [ 0.18431373,  0.19215687,  0.13725491],
         [ 0.16862746,  0.17647059,  0.12156863]],

        ..., 
        [[ 0.66274512,  0.55686277,

In [0]:
# making predictions
prediction = model_final.predict(test)

In [0]:
leaderboard_test_pred['has_oilpalm'] = prediction

In [0]:
leaderboard_test_pred.has_oilpalm.unique()

In [0]:
leaderboard_test_pred[leaderboard_test_pred.has_oilpalm == 1]

# Leaderboard Holdback Data Prediction

In [0]:
holdpath = holdFolderPath
holdimagesList = sorted(os.listdir(holdpath))
holdimagesList

In [0]:
leaderboard_hold_pred = pd.DataFrame(holdimagesList , columns=['image_id'])
leaderboard_hold_pred

In [0]:
hold_image = []
for i in tqdm(range(len(holdimagesList))):
    img = image.load_img(holdImagesPath + holdimagesList[i], target_size=(224,224,3))
    img = image.img_to_array(img)
    img = img/255
    hold_image.append(img)
hold = np.array(hold_image)
hold

In [0]:
# making predictions
prediction = model_final.predict(hold)

In [0]:
leaderboard_hold_pred['has_oilpalm'] = prediction

In [0]:
leaderboard_hold_pred.has_oilpalm.unique()

In [0]:
leaderboard_test_pred[leaderboard_test_pred.has_oilpalm == 1]

# Building submission file

In [0]:
leaderboard_test_pred.shape

In [0]:
leaderboard_hold_pred.shape

In [0]:
finalSubmission = leaderboard_test_pred.append(leaderboard_hold_pred)
finalSubmission.shape

In [0]:
finalSubmission.head()

In [0]:
finalSubmission.to_csv(subFilePath,index=False)